# 【課題】量子機械学習を使って、新しい素粒子現象の発見を目指す

以下の実習「素粒子現象の探索への応用」で行った手法を、どういう方法でも良い (ただし量子回路を使った手法に限る）ので、改良する方法を考えてください（ただし、「データサンプルの事象数を増やす」方法は除きます）。

例えば:

- 入力変数の選び方（数やどの変数を使うか）
- 特徴量マップの構造（変数変換の方法やゲートの選び方など）
- 変分フォームの構造（ゲートの選び方、回路の深さ、エンタングルの方法など）
- 変分量子回路以外の方法でもOK

ここでは、「改良」の定義は

- 訓練用データでのAUC値の向上
- テスト用データでのAUC値の向上

とします。

**提出方法は自由です**。
- 改良した結果のプロットやコードなど（以下のNotebookでコードセルを追加・修正などして出してもらっても良いですし、全く新しく作ってもらっても構いません）
- テキストに書いた考察でもOK（ただし、なぜそう考えるか理由を添えてください)

**注**：使うコードがqiskitの古いバージョンに対応したものなので、いろいろWarningが出ると思いますが無視してもらって大丈夫です。

## 素粒子現象の探索への応用

In [1]:
# Tested with python 3.7.9, qiskit 0.23.5, numpy 1.20.1
import logging
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute
from qiskit.circuit.library import TwoLocal, ZFeatureMap, ZZFeatureMap
from qiskit.aqua import QuantumInstance, set_qiskit_aqua_logging
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.algorithms.optimizers import SPSA, COBYLA

#set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

/Users/terashi/miniconda3/lib/python3.6/site-packages/qiskit/__init__.py:67: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of the 0.17.0 release. Support for running Qiskit with Python 3.6 will be removed in a future release.
  "future release.", DeprecationWarning)


### 学習データの準備<a id='susy_data'></a>

In [ ]:
# ファイルから変数を読み出す
df = pd.read_csv("data/SUSY_1K.csv",
                 names=('isSignal','lep1_pt','lep1_eta','lep1_phi','lep2_pt','lep2_eta',
                        'lep2_phi','miss_ene','miss_phi','MET_rel','axial_MET','M_R','M_TR_2',
                        'R','MT2','S_R','M_Delta_R','dPhi_r_b','cos_theta_r1'))

# 学習に使う変数の数
feature_dim = 3  # dimension of each data point

# 3, 5, 7変数の場合に使う変数のセット
if feature_dim == 3:
    SelectedFeatures = ['lep1_pt', 'lep2_pt', 'miss_ene']
elif feature_dim == 5:
    SelectedFeatures = ['lep1_pt','lep2_pt','miss_ene','M_TR_2','M_Delta_R']
elif feature_dim == 7:
    SelectedFeatures = ['lep1_pt','lep1_eta','lep2_pt','lep2_eta','miss_ene','M_TR_2','M_Delta_R']

# 学習に使う事象数: trainingは訓練用サンプル、testingはテスト用サンプル
training_size = 20
testing_size = 20

# オプティマイザーをCallする回数の上限
niter = 100

random_seed = 10598

df_sig = df.loc[df.isSignal==1, SelectedFeatures]
df_bkg = df.loc[df.isSignal==0, SelectedFeatures]

# サンプルの生成
df_sig_training = df_sig.values[:training_size]
df_bkg_training = df_bkg.values[:training_size]
df_sig_test = df_sig.values[training_size:training_size+testing_size]
df_bkg_test = df_bkg.values[training_size:training_size+testing_size]
training_input = {'1':df_sig_training, '0':df_bkg_training}
test_input = {'1':df_sig_test, '0':df_bkg_test}
#print('train_input =',training_input)
#print('test_input =',test_input)

datapoints, class_to_label = split_dataset_to_data_and_labels(test_input)
datapoints_tr, class_to_label_tr = split_dataset_to_data_and_labels(training_input)

### 量子状態の生成<a id='susy_state_preparation'></a>

In [ ]:
feature_map = ZFeatureMap(feature_dim, reps=1)

### 変分フォームを使った状態変換<a id='susy_variational_form'></a>

In [ ]:
two = TwoLocal(feature_dim, ['ry','rz'], 'cz', 'full', reps=1)
two.draw('mpl')

### 測定とモデル出力<a id='susy_measurement'></a>

In [ ]:
# シミュレータで実行する場合
backend = Aer.get_backend('qasm_simulator')

# 量子コンピュータで実行する場合
#from qiskit import IBMQ
#IBMQ.enable_account('__your_token__')
#provider0 = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
#backend_name = 'ibmq_santiago'
#backend = provider0.get_backend(backend_name)


optimizer = COBYLA(maxiter=niter, disp=True)
vqc = VQC(optimizer, feature_map, two, training_input, test_input)

quantum_instance = QuantumInstance(backend=backend, shots=1024,
                                   seed_simulator=random_seed, seed_transpiler=random_seed,
                                   skip_qobj_validation=True)

result = vqc.run(quantum_instance)
print(" --- Testing success ratio: ", result['testing_accuracy'])

In [ ]:
# 結果を求める
predicted_probs, predicted_labels = vqc.predict(datapoints[0])
prob_test_signal = predicted_probs[:,1]
#predicted_classes = map_label_to_class_name(predicted_labels, vqc.label_to_class)
#print(" ---   Prediction:   {}".format(predicted_classes))
predicted_probs_tr, predicted_labels_tr = vqc.predict(datapoints_tr[0])
prob_train_signal = predicted_probs_tr[:,1]

# ROC曲線を描いてみる
from sklearn.metrics import roc_curve, auc, roc_auc_score
fpr, tpr, thresholds = roc_curve(datapoints[1], prob_test_signal, drop_intermediate=False)
fpr_tr, tpr_tr, thresholds_tr = roc_curve(datapoints_tr[1], prob_train_signal, drop_intermediate=False)
roc_auc = auc(fpr, tpr)
roc_auc_tr = auc(fpr_tr, tpr_tr)

plt.plot(fpr, tpr, color='darkorange', lw=2, label='Testing Data (AUC = %0.3f)' % roc_auc)
plt.plot(fpr_tr, tpr_tr, color='darkblue', lw=2, label='Training Data (AUC = %0.3f)' % roc_auc_tr)
plt.plot([0, 0], [1, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()
